In [1]:
from nilearn import connectome
import pickle
import numpy as np
import glob
import pandas as pd
import numpy as np

In [2]:
base_path = "D:/datasets/ABIDE"
data_path = "D:/datasets/ABIDE/Outputs/dparsf/filt_global/rois_aal"

In [3]:
phenotypic = pd.read_csv(f"{base_path}/Phenotypic_V1_0b_preprocessed1.csv")

In [4]:
data = []
labels = []
fc_data = []
fc_labels = []
for i, sub in phenotypic.iterrows():
    filename = sub["FILE_ID"]
    file = glob.glob(f"{data_path}/{filename}*")
    if file:
        time_series = pd.read_table(file[0]).to_numpy()
        data.append(np.append(time_series, np.full((315-time_series.shape[0],116), np.nan), axis=0))
        labels.append([sub["DX_GROUP"]-1])
        fc = connectome.ConnectivityMeasure(kind="correlation").fit_transform([time_series])[0].reshape((116, 116, 1))
        fc_data.append(fc)
        fc_labels.append([sub["DX_GROUP"]-1])

In [5]:
np.savez(f"{base_path}/ABIDE_dataset", data=data, labels=labels)
np.savez(f"{base_path}/ABIDE_FC_dataset", data=fc_data, labels=fc_labels)

数据增强

In [17]:
def sliceWindows(time_series, frame, interval):
    """ 切割滑动窗口 """
    """
        time_series: numpy.ndarray, 需要切割的时间序列
        frame: int, 每帧窗口尺寸
        interval: int, 窗口间步长间隔
    """
    if len(time_series) < frame:
        return [time_series]
    windows = []
    l, r = 0, frame
    while r < time_series.shape[0]:
        windows.append(time_series[l:r])
        l += interval
        r = l + frame
    return windows

In [18]:
data = []
labels = []
fc_data = []
fc_labels = []
for i, sub in phenotypic.iterrows():
    filename = sub["FILE_ID"]
    file = glob.glob(f"{data_path}/{filename}*")
    if file:
        time_series = pd.read_table(file[0]).to_numpy()
        for window in sliceWindows(time_series, 60, 10):
            data.append(window)
            labels.append([sub["DX_GROUP"]-1])
            fc = connectome.ConnectivityMeasure(kind="correlation").fit_transform([window])[0].reshape((116, 116, 1))
            fc_data.append(fc)
            fc_labels.append([sub["DX_GROUP"]-1])

In [19]:
np.savez(f"{base_path}/ABIDE_augmented_dataset", data=data, labels=labels)
np.savez(f"{base_path}/ABIDE_FC_augmented_dataset", data=fc_data, labels=fc_labels)